In [1]:
from interface_generator import core, print_near_axis, convert_vector_index,d_hkl,three_dot,cross_plane, get_right_hand, write_trans_file, get_ang_list,super_cell,cell_expands,shift_termi_left,adjust_orientation,POSCAR_to_cif,get_R_to_screen,terminates_scanner_left,get_height,excess_volume,surface_vacuum,write_POSCAR,terminates_scanner_right
from csl_generator import print_list,get_theta_m_n_list, degrees
import numpy as np
import glob
import shutil
import os
from cellcalc import MID, DSCcalc, get_primitive_hkl, get_right_hand, find_integer_vectors, get_pri_vec_inplane, get_ortho_two_v, ang, search_MI_n, get_normal_from_MI, match_rot
from numpy import array, dot, round, cross, ceil
from numpy.linalg import inv, det, norm
from numpy import cross, dot, ceil
from numpy.linalg import norm, inv
import math
import matplotlib.pyplot as plt
from abc import ABCMeta, abstractclassmethod
import copy
import itertools
import collections
from numpy import dot, cross, ceil, floor, cos, sin, tile, array, arange, meshgrid, delete, column_stack, eye

In [2]:
class Element3:
    def __init__(self,name,dirname,crystal_structure,atoms_type_list,atoms_charge_list):
        self.name = name
        self.dirname = dirname
        self.grandenergy = self.getLatE(dirname)
        self.LP = self.getLatP(dirname)
        self.mass = self.get_mass(dirname)
        self.ciffile = glob.glob(f"{dirname}/*.cif")[0]
        self.crystal_structure = crystal_structure # this is a class representing crystal structure
        self.atoms_type_list = atoms_type_list
        self.atoms_charge_list = atoms_charge_list
#         with open(f"{self.dirname}/proto.in","r") as f:
#             self.proto_text = f.read() 
    
    def potential(self):
        for dirs in glob.glob(f"{self.dirname}/potential/*"):
            shutil.copy(f"{dirs}",f"{os.getcwd()}")
        
#     def proto(self):
#         shutil.copy(f"{self.dirname}/proto.in",f"{os.getcwd()}")
    
    @staticmethod
    def getLatE(dirname):
        """
        read the Energy of single crystal 
        """
        with open(f'{dirname}/atomsout','r') as f:
            lines=f.readlines()
            LatE = lines[-4:][0].split()[4].replace(';', '')
            
        with open(f'{dirname}/atomsout','r') as f:
            lines=f.readlines()
            LatE_one = lines[-4:][0].split()[5].replace(';', '')
    
        with open(f'{dirname}/atomsout','r') as f:
            lines=f.readlines()
            LatE_two = lines[-3:][0].split()[5].replace(';', '')
    
        return float(LatE_one), float(LatE_two)

    @staticmethod
    def getLatP(dirname):
        """
        read the Lattice Parameter
        """ 
        with open(f'{dirname}/atomsout','r') as f:
            lines=f.readlines()
            LatP = lines[-6:][0].split()[4].replace(';', '')
    
        return float(LatP)

    @staticmethod
    def get_mass(dirname):
        with open(f'{dirname}/mass','r') as f:
            lines=f.readlines()
        return float(lines[0]), float(lines[1])
    

In [3]:
class ProtoinCreator(metaclass=ABCMeta):
    # element →　Element クラス　オブジェクト　インスタンス化"前"
    # SanplingMethod　→　SamplingMethodCreator クラス　オブジェクト　インスタンス化"前"
    def __init__(self,element,SamplingMethod):
        self.element = element
        self.sampling_method = SamplingMethod
        self.vanishing_rule = element.crystal_structure.vanishing_rule # this is function
#         self.crystal_structurte = self.element.crystal_structure
        # で、僕はこのself.crystal_structureが持っているメソッドを使いたい
    def generate(self,axis,hkl):
        samplingmethod = self.sampling_method(axis,hkl) # インスタンス化後のSamplingMethodCreatorクラスメソッド
        samplingmethod.vanishing_rule = self.vanishing_rule # passしていたvanishing_ruleをオーバーライド
        protoinbodycreator = ProtoinBodyCreator(samplingmethod)
        self.body_text = protoinbodycreator.generate(self.element.name)
    # サブクラスがインスタンス化されるまで保留して一旦設計を進めることができる
    
    @abstractclassmethod
    def header(self):
        pass
    
    @abstractclassmethod
    def body(self):
        pass 
    
    
    #テンプレートの役割を持つクラス
    def create(self):
        h = self.header()
        b = self.body()
        return f"{h}\n\n{b}"

In [4]:
class Protoin(ProtoinCreator):
    
    # 継承先を固定しておけばテンプレートを動かさなくていい
    def header(self):
        mass_one, mass_two = self.element.mass
        grandenergy_one,grandenergy_two = self.element.grandenergy
        LP = self.element.LP
        text = f"""


variable element_mass_one equal {mass_one} 

variable element_mass_two equal {mass_two} 

variable minimumenergy_one equal {grandenergy_one}

variable minimumenergy_two equal {grandenergy_two}

variable lattice_param equal {LP}
                """
        
        return text

    
    def body(self):
        return self.body_text
    

In [5]:
class ProtoinBodyCreator:
    def __init__(self,sampling_method_creator):
        self.sampling = sampling_method_creator
        
    def generate(self,element):
        return self.sampling.calc_condition(element)

In [6]:
class Calc_Condition:
    def __init__(self,element,SamplingMethod,cell_size,fix_bulk_region,middle_region,start_bulk,end_bulk,vaccum):

#         self.cnid_sampling = cnid_sampling
        self.element = element
#        self.protoin = Protoin(self.element,SamplingMethod)
        self.protoin = Protoin(self.element,SamplingMethod)
        self.cell_size = cell_size
        self.fix_bulk_region = fix_bulk_region
        self.middle_region = middle_region
        self.start_bulk = start_bulk
        self.end_bulk = end_bulk
        self.vaccum = vaccum
        
    def define_bicrystal_regions(self,xhi):
        tol = 1e-5
        """
        generate a file defining some regions in the LAMMPS and define the atoms
        inside these regions into some groups.
        argument:
        region_names --- list of name of regions
        region_los --- list of the low bounds
        region_his --- list of the hi bounds
        """
        end_fixbulk1 = xhi/2-self.fix_bulk_region
        start_fixbulk2 = xhi/2+self.fix_bulk_region
        start_middle = xhi/2-self.middle_region
        end_middle = xhi/2+self.middle_region
        start_right = xhi/2 - tol
        start_bulk = self.start_bulk
        end_bulk = self.end_bulk


        with open('blockfile', 'w') as fb:
            fb.write('region fixbulk1 block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(end_fixbulk1))
            fb.write('region fixbulk2 block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
            format(start_fixbulk2))
            fb.write('region middle block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_middle,end_middle))
            fb.write('region right block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
            format(start_right))
            fb.write('region left block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_right))
            fb.write('region bulk block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_bulk,end_bulk))
            fb.write('group fixbulk1 region fixbulk1 \n')
            fb.write('group fixbulk2 region fixbulk2 \n')
            fb.write('group middle region middle \n')
            fb.write('group right region right \n')
            fb.write("group left region left \n")
            fb.write('group bulk region bulk \n')
            fb.write("variable startright equal {0:.16f}".\
            format(start_right))
        
    def define_bicrystal_regions2(self,xhi):
        half_cell = xhi/2
        text = f"""

    displace_atoms right move ${{tx}} ${{ty}} ${{tz}} units box

    group left_dash region left 

    group deleted_atoms subtract left_dash left

    displace_atoms deleted_atoms move {half_cell} 0 0 units box

    group fixbulk1_dash region fixbulk1 
    group fixbulk2_dash region fixbulk2 
    group middle_dash region middle 
    group bulk_dash region bulk
                """
        with open("blockfile2", "w") as f:
            f.writelines(text)
        
    def generate_files(self,axis,hkl,xhi):
        self.protoin.generate(axis,hkl)
        self.element.potential()
        self.define_bicrystal_regions(xhi)
        self.define_bicrystal_regions2(xhi)
        with open("proto.in","w") as f:
            f.writelines(self.protoin.create())
#        atoms_type_list = self.element.atoms_type_list
#        atoms_charge_list = self.element.atoms_charge_list
#        add_charge_to_atominfile(type_list=atoms_type_list,charge_list=atoms_charge_list,filename="atominfile")
        
        
#         if self.cnid_sampling==True:
#             self.element.proto()
#         else:
#             with open("proto.in","w") as f:
#                 f.writelines(self.protoin.create())

In [31]:
def write_LAMMPS2(lattice, atoms, elements, filename = 'lmp_atoms_file', orthogonal = False,type_list=[],charge_list=[],vaccum = 20):
    """
    write LAMMPS input atom file file of a supercell
    argument:
    lattice --- lattice matrix
    atoms --- fractional atoms coordinates
    elements --- list of element name of the atoms
    orthogonal --- whether write orthogonal cell
    """

    #list of elements
    element_species = np.unique(elements)

    #to Cartesian
    atoms = dot(lattice, atoms.T).T

    #get the speicie identifiers (e.g. 1 for Cu, 2 for O...)
    element_indices = np.arange(len(element_species)) + 1
    species_identifiers = np.arange(len(atoms))

    for i in range(len(element_species)):
        indices_this_element = np.where(elements == element_species[i])[0]
        species_identifiers[indices_this_element] = element_indices[i]

    species_identifiers = np.array([species_identifiers])

    #the atom ID
    IDs = np.arange(len(atoms)).reshape(1, -1) + 1

    #get the final format
    Final_format = np.concatenate((IDs.T, species_identifiers.T), axis=1)
    Final_format = np.concatenate((Final_format, atoms), axis = 1)

    #define the box
    #xlo, xhi
    xhi, yhi, zhi = lattice[0][0], lattice[1][1], lattice[2][2]
    xlo, ylo, zlo = 0, 0, 0
    xy = lattice[:,1][0]
    xz = lattice[:,2][0]
    yz = lattice[:,2][1]

    xhi_dash = xhi + vaccum
    final_text = add_charge(Final_format,xhi_dash,yhi,zhi,type_list,charge_list)
    
    with open(filename, 'w') as f:
        f.write('#LAMMPS input file of atoms generated by interface_master. The elements are: ')
        for i in range(len(element_indices)):
            f.write('{0} {1} '.format(element_indices[i], element_species[i]))
        f.write('\n {} atoms \n \n'.format(len(atoms)))
        f.write('{} atom types \n \n'.format(len(element_species)))
        f.write('{0:.8f} {1:.8f} xlo xhi \n'.format(xlo, xhi_dash))
        f.write('{0:.8f} {1:.8f} ylo yhi \n'.format(ylo, yhi))
        f.write('{0:.8f} {1:.8f} zlo zhi \n\n'.format(zlo, zhi))
        if orthogonal == False:
            f.write('{0:.8f} {1:.8f} {2:.8f} xy xz yz \n\n'.format(xy, xz, yz))
        f.write('Atoms \n \n')
        f.write(f"{final_text}")
#         np.savetxt(f, Final_format, fmt='%i %i %.16f %.16f %.16f')
    f.close()

In [32]:
def add_charge_to_atominfile(type_list, charge_list,filename="atominfile"):
    a, b, c, atoms = read_atominfile(f"{filename}")
    final_text = add_charge(atoms,a,b,c,type_list,charge_list)
    dim = np.array([1,1,1])
    X = atoms.copy()

    NumberAt = len(X) 

    dimx, dimy, dimz = dim

    xlo = 0.00000000
    xhi = a + 20
    ylo = 0.00000000
    yhi = b
    zlo = 0.00000000
    zhi = c

    yz = 0.0


    with open(filename, 'w') as f:
        f.write('#Header \n \n')
        f.write('{} atoms \n \n'.format(NumberAt))
        f.write('2 atom types \n \n')
        f.write('{0:.8f} {1:.8f} xlo xhi \n'.format(xlo, xhi))
        f.write('{0:.8f} {1:.8f} ylo yhi \n'.format(ylo, yhi))
        f.write('{0:.8f} {1:.8f} zlo zhi \n\n'.format(zlo, zhi))
        f.write('{0:.8f} {1:.8f} {2:.8f} xy xz yz \n\n'.format(0, 0, yz))            
        f.write('Atoms \n \n')
        f.write(f"{final_text}")
    f.close()

In [33]:
class core2(core):
    def __init__(self, file_1, file_2):
        super().__init__(file_1,file_2)
    def compute_stgb(self, axis, plane_n, lim = 20, tol = 1e-10):
        """
        compute the transformation to obtain the supercell of the two slabs forming a interface
        argument:
        hkl --- miller indices of the plane expressed in lattice_1
        lim --- the limit searching for a CSL vector cross the plane
        normal_ortho --- whether limit the vector crossing the GB to be normal to the GB
        plane_ortho --- whether limit the two vectors in the GB plane to be orthogonal
        tol --- tolerance judging whether orthogonal
        """
        hkl = MID(self.lattice_1,plane_n)
        self.d1 = d_hkl(self.lattice_1, hkl)
        lattice_2 = three_dot(self.R, self.D, self.lattice_2)
        hkl_2 = get_primitive_hkl(hkl, self.lattice_1, lattice_2)
        self.d2 = d_hkl(lattice_2, hkl_2)
        hkl_c = get_primitive_hkl(hkl, self.lattice_1, self.CSL) # miller indices of the plane in CSL
        hkl_c = np.array(hkl_c)
        
        #plane_n = dot(self.lattice_1,hkl)
        axis_cartesian = dot(self.conv_lattice_1,axis)
        v1 = cross_plane(lattice=self.CSL, n=plane_n, lim=lim,  orthogonal=True, tol=tol) # a CSL basic vector cross the plane
        v2 = cross_plane(lattice=self.CSL, n=axis_cartesian, lim=lim, orthogonal=True, tol=tol)
        v3_cartesian = cross(v1,v2)
        v3 = cross_plane(lattice=self.CSL, n=v3_cartesian, lim=lim, orthogonal=True, tol=tol)
        supercell = np.column_stack((v1,v2,v3)) # supercell of the bicrystal
        supercell = get_right_hand(supercell) # check right-handed


        self.bicrystal_U1 = np.array(np.round(dot(inv(self.lattice_1), supercell),8),dtype = int)
        self.bicrystal_U2 = np.array(np.round(dot(inv(self.lattice_2_TD), supercell),8),dtype = int)
        self.cell_calc.compute_CNID(hkl)
        CNID = self.cell_calc.CNID
        self.CNID = dot(self.lattice_1, CNID)
    def get_bicrystal(self, dydz = np.array([0.0,0.0,0.0]), dx = 0, dp1 = 0, dp2 = 0, \
                      xyz_1 = [1,1,1], xyz_2 = [1,1,1], vx = 0, filename = 'POSCAR', \
                      two_D = False, filetype = 'VASP', LAMMPS_file_ortho = False, mirror = False,atoms_type_list=[],atoms_charge_list=[],vaccum=20):
        """
        generate a cif file for the bicrystal structure
        argument:
        dydz --- translation vector in the interface
        dx --- translation normal to the interface
        dp1 --- termination of slab 1
        dp2 --- termination of slab 2
        xyz --- expansion
        two_D --- whether a two CSL
        LAMMPS_file_ortho --- whether the output LAMMPS has orthogonal cell
        """
        #get the atoms in the primitive cell
        lattice_1, atoms_1, elements_1 = self.lattice_1.copy(), self.atoms_1.copy(), self.elements_1.copy()
        lattice_2, atoms_2, elements_2 = self.lattice_2.copy(), self.atoms_2.copy(), self.elements_2.copy()

        # deform & rotate lattice_2
        if two_D == True:
            lattice_2 = dot(self.a2_transform, lattice_2)
        else:
            lattice_2 = three_dot(self.R, self.D, lattice_2)
        #make supercells of the two slabs
        atoms_1, elements_1, lattice_1 = super_cell(self.bicrystal_U1, \
                                                    lattice_1, atoms_1, elements_1)

        if mirror == False:
            atoms_2, elements_2, lattice_2 = super_cell(self.bicrystal_U2, \
                                                lattice_2, atoms_2, elements_2)
        else:
            atoms_2, elements_2, lattice_2 = atoms_1.copy(), elements_1.copy(), lattice_1.copy()
            atoms_2[:,0] = - atoms_2[:,0] + 1
            atoms_c, elements_c = atoms_2.copy(), elements_2.copy()
            elements_c = elements_c[atoms_c[:,0] + 0.000001 > 1]
            atoms_c = atoms_c[atoms_c[:,0] + 0.000001 > 1]
            atoms_c[:,0] = atoms_c[:,0] - 1
            atoms_2 = np.vstack((atoms_2, atoms_c))
            elements_2 = np.append(elements_2, elements_c)

            # delete overwrapping atoms
            elements_2 = elements_2[np.where(atoms_2[:,0] + 0.000001 < 1)]
            atoms_2 = atoms_2[atoms_2[:,0] + 0.000001 < 1]
            
        
        #expansion
        if not (np.all(xyz_1 == 1) and np.all(xyz_2 == 1)):
            if not np.all([xyz_1[1], xyz_1[2]] == [xyz_2[1], xyz_2[2]]):
                raise RuntimeError('error: the two slabs must expand to the same dimension in the interface plane')
            lattice_1, atoms_1, elements_1 = cell_expands(lattice_1, atoms_1, \
                                                          elements_1, xyz_1)
            lattice_2, atoms_2, elements_2 = cell_expands(lattice_2, atoms_2, \
                                                          elements_2, xyz_2)

        #termination
        if dp1 != 0:
            atoms_1, elements_1 = shift_termi_left(lattice_1, dp1, atoms_1, elements_1)
        if dp2 != 0:
            atoms_2, elements_2 = shift_termi_right(lattice_2, dp2, atoms_2, elements_2)


        #adjust the orientation
        lattice_1, self.orient = adjust_orientation(lattice_1)
        lattice_2 = dot(self.orient, lattice_2)
        
        write_POSCAR(lattice_1, atoms_1, elements_1, 'POSCAR')
        POSCAR_to_cif('POSCAR','cell_1.cif')
        write_POSCAR(lattice_2, atoms_2, elements_2, 'POSCAR')
        POSCAR_to_cif('POSCAR','cell_2.cif')
        os.remove('POSCAR')

        self.R_see_plane = get_R_to_screen(lattice_1)

        self.plane_list_1, self.elements_list_1, self.indices_list_1, self.dp_list_1 \
         = terminates_scanner_left(lattice_1, atoms_1, elements_1, self.d1)

        self.plane_list_2, self.elements_list_2, self.indices_list_2, self.dp_list_2 \
         = terminates_scanner_right(lattice_2, atoms_2, elements_2, self.d2)

        self.slab_lattice_1 = lattice_1.copy()
        self.slab_lattice_2 = lattice_2.copy()

        #combine the two lattices and translate atoms
        lattice_bi = lattice_1.copy()
        if two_D:
            height_1 = get_height(lattice_1)
            height_2 = get_height(lattice_2)
            lattice_bi[:,0] = lattice_bi[:,0] * (1 + height_2 / height_1)
            #convert to cartesian
            atoms_1 = dot(lattice_1, atoms_1.T).T
            atoms_2 = dot(lattice_2, atoms_2.T).T
            #translate
            atoms_2 = atoms_2 + lattice_1[:,0]
            #back to the fractional coordinates
            atoms_1 = dot(inv(lattice_bi), atoms_1.T).T
            atoms_2 = dot(inv(lattice_bi), atoms_2.T).T
        else:
            lattice_bi[:,0] = 2 * lattice_bi[:,0]
            atoms_1[:,0] = atoms_1[:,0] / 2
            atoms_2[:,0] = (atoms_2[:,0] + 1) / 2

        #excess volume
        if dx != 0:
            excess_volume(lattice_1, lattice_bi, atoms_1, atoms_2, dx)

        #in-plane translation
        if norm(dydz) > 0:
            dydz = dot(self.orient, dydz)
            plane_shift = dot(inv(lattice_bi), dydz)
            atoms_2 = atoms_2 + plane_shift

        #combine the two slabs
        elements_bi = np.append(elements_1, elements_2)
        atoms_bi = np.vstack((atoms_1, atoms_2))

        #wrap the periodic boundary condition
        atoms_bi[:,1] = atoms_bi[:,1] - floor(atoms_bi[:,1])
        atoms_bi[:,2] = atoms_bi[:,2] - floor(atoms_bi[:,2])
        #vacummn
        if vx > 0:
            surface_vacuum(lattice_1, lattice_bi, atoms_bi, vx)

        #save
        self.lattice_bi = lattice_bi
        self.atoms_bi = atoms_bi
        self.elements_bi = elements_bi

        if filetype == 'VASP':
            write_POSCAR(lattice_bi, atoms_bi, elements_bi, filename)
        elif filetype == 'LAMMPS':
            write_LAMMPS2(lattice_bi, atoms_bi, elements_bi, filename, LAMMPS_file_ortho,atoms_type_list,atoms_charge_list,vaccum)
        else:
            raise RuntimeError("Sorry, we only support for 'VASP' or 'LAMMPS' output")

In [34]:
  class MyInterface:
    def __init__(self,max_sigma,axis,calc_condition):
        self.calc_condition = calc_condition
        self.cif_filename = self.calc_condition.element.ciffile
        self.LP = self.calc_condition.element.LP
        self.max_sigma = max_sigma
        self.axis = axis
        
        self.my_interface = core2(self.cif_filename, self.cif_filename)
        
        self.my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=self.max_sigma, sgm2=self.max_sigma, dd = 1e-4)
        factor = self.LP/(norm(self.my_interface.conv_lattice_1[:,0]))
        self.my_interface.lattice_1 =  self.my_interface.lattice_1*factor
        self.my_interface.lattice_2 =  self.my_interface.lattice_2*factor
        self.my_interface.conv_lattice_1 =  self.my_interface.conv_lattice_1*factor
        self.my_interface.conv_lattice_2 =  self.my_interface.conv_lattice_2*factor
        

In [35]:
class Factory(object): # ここはインスタンスを生成する工場だから普段インスタンスを作ることをやる
    def create(self,hkl,delivery_point="."): # インスタンスをcreate
        # 回転軸、回転角、plane_nを受け取って作成
        axis, theta, plane_n = self.get_gb_info_from_hkl(hkl)
        self.createGB(axis,theta,plane_n,hkl,delivery_point)
    
    def createGB(self,axis,theta,plane_n,hkl,delivery_point):
        # interfaceインスタンス化 + get_gbfiles実行
        pass

    
    def generate_hkl_list(self,axis,max_sigma):
        pass
    
    def get_gb_info_from_hkl(self,hkl):
        pass
    


In [36]:
class GBFactory(Factory):
    def __init__(self,max_sigma,axis,calc_condition):
        self.calc_condition = calc_condition
        self.element = self.calc_condition.element
        self.cif_filename = self.calc_condition.element.ciffile
        self.LP = self.calc_condition.element.LP
        self.max_sigma = max_sigma
        self.axis = axis
        self.primitive_interface = MyInterface(max_sigma,axis,calc_condition).my_interface
        self.conventional_interface = MyInterface(max_sigma,axis,calc_condition).my_interface
        
    def createGB(self,axis,theta,plane_n,hkl,delivery_point):
    # interface serch csl create_gbfiles実行
        axis_cart = dot(self.primitive_interface.conv_lattice_1, np.array(axis))

        axis_pri = np.array(np.round(dot(inv(self.primitive_interface.lattice_1), axis_cart),8),dtype = int)
#         print(axis_pri)
#         print(theta)
#         print(self.primitive_interface.lattice_1)
        
        self.primitive_interface.search_one_position(axis_pri,theta-0.0000001,1,0.0000001)
        self.get_gb_files(axis,plane_n,hkl,delivery_point)
        
#         get_gb_files(self,interface,axis,hkl,calc_condition,input_dirname=None)
    
    @classmethod
    def generate_hkl_list(cls,axis,max_sigma,calc_condition):
        conventional_interface = copy.deepcopy(cls(max_sigma,axis,calc_condition).conventional_interface)
        conventional_interface.lattice_1 = conventional_interface.conv_lattice_1
        conventional_interface.lattice_2 = conventional_interface.conv_lattice_2
        
        thetas = cls.get_thetas(axis,max_sigma)

        gb_dict = {}

        for theta in thetas:

            conventional_interface.search_one_position(np.array(axis),theta-0.0000001,1,0.0000001)
            n_1, n_2 = cls.get_a_b(conventional_interface.CSL, dot(conventional_interface.lattice_1,np.array(axis)))
            
            hkl_name = np.array(np.round(dot(inv(conventional_interface.conv_lattice_1), n_1),8),dtype = int)
            hkl_name = np.array(hkl_name, dtype=int)
            hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
            hkl_x, hkl_y, hkl_z = hkl_name
            axis_x, axis_y, axis_z = axis

            gb_info1 = {f"{hkl_x}_{hkl_y}_{hkl_z}":[axis,theta,n_1]}
            print(hkl_name)

            hkl_name = np.array(np.round(dot(inv(conventional_interface.conv_lattice_1), n_2),8),dtype = int)
            hkl_name = np.array(hkl_name, dtype=int)
            hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
            hkl_x, hkl_y, hkl_z = hkl_name
            axis_x, axis_y, axis_z = axis

            gb_info2 = {f"{hkl_x}_{hkl_y}_{hkl_z}":[axis,theta,n_2]}

            if axis==[1,1,1,]:
                gb_dict = {**gb_dict,**gb_info1}
            else:
                gb_dict = {**gb_dict,**gb_info1,**gb_info2}
            
            print(list(gb_dict.keys()))
        
        cls.gb_dict = gb_dict
        
        return cls(max_sigma,axis,calc_condition)
    
    def get_gb_info_from_hkl(self,hkl):
        return self.gb_dict[hkl]
     
    @staticmethod
    def get_thetas(axis,max_sigma):
        theta_list = []
        sigma_list = []
        lists, thetas = GBFactory.getsigmas(axis, max_sigma)
        return thetas
    
    @staticmethod
    def get_csl_basis(CSL,axis):
        tol = 1e-8
        dot_list = get_ang_list(CSL.T,axis)
        normal_v = CSL.T[np.where(abs(dot_list - 1) > tol)[0]]
        return normal_v
    
    @staticmethod
    def get_a_b(CSL, axis):
        hkl_perp_axis = MID(CSL, axis)
        a, b = get_pri_vec_inplane(hkl_perp_axis, CSL).T
        
        if (norm(cross(axis,[1,1,1])) < 1e-8):
            v1, v2 = GBFactory.get_csl_basis(CSL,axis)
            if dot(v1,v2) < 0:
                v2 = -v2
        
        if(norm(cross(axis,[1,0,0])) < 1e-8):
            b = a + b
        elif (norm(cross(axis,[1,1,1])) < 1e-8):
            a = v1 + v2
#         elif (norm(cross(axis,[1,1,1])) < 1e-8):
#             if dot(a,b) < 0:
#                 b = a + b
#             b = a + b
#         if (abs(norm(a) - norm(b)) < 1e-8):
#             raise RuntimeError ('the tow vectors are identical!')

        return a.T, b.T
    
    @staticmethod
    def getsigmas(uvw, limit):
        """
        ---by Haddian---
        prints a list of smallest sigmas/angles for a given axis(uvw).
        """
        sigmas = []
        thetas = []
        for i in range(limit):

            tt = get_theta_m_n_list(uvw, i)
            if len(tt) > 0 and i > 1:
                theta, m, n = tt[0]
                sigmas.append(i)
                thetas.append(degrees(theta))                
        return sigmas, thetas
    
    @staticmethod
    def get_expansion_xyz(cell,cell_size):
        cell_x, cell_y, cell_z = cell_size
        exp_x = ceil(cell_x/norm(cell[:,0]))
        exp_y = ceil(cell_y/norm(cell[:,1]))
        exp_z = ceil(cell_z/norm(cell[:,2]))
        return exp_x, exp_y, exp_z
    
    def get_gb_files(self,axis,plane_n,hkl,delivery_point):
        basedir = os.getcwd()
        self.primitive_interface.compute_stgb(axis, plane_n, lim = 10, tol = 1e-5)
        half_cell = dot(self.primitive_interface.lattice_1, self.primitive_interface.bicrystal_U1)
        x,y,z = self.get_expansion_xyz(half_cell,self.calc_condition.cell_size)
        xhi = 2*x*norm(half_cell[:,0])


        hkl_intlist = list(map(int,hkl.split("_")))
        hkl_x, hkl_y, hkl_z = hkl_intlist
        axis_x, axis_y, axis_z = axis
        dirname = f"{axis_x}{axis_y}{axis_z}_{hkl_x}_{hkl_y}_{hkl_z}_gb"

        final_dirname = os.path.join(delivery_point,dirname)
        os.makedirs(final_dirname,exist_ok=True)
        os.chdir(final_dirname) 

        self.primitive_interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = False,atoms_type_list=self.element.atoms_type_list,atoms_charge_list=self.element.atoms_charge_list,vaccum=self.calc_condition.vaccum)

        CNID = dot(self.primitive_interface.orient, self.primitive_interface.CNID)
        v1 = np.array([0,1.,0])*CNID[:,0][1] + np.array([0,0,1.])*CNID[:,0][2]
        v2 = np.array([0,1.,0])*CNID[:,1][1] + np.array([0,0,1.])*CNID[:,1][2]

        n1 = int(ceil(norm(v1)/0.2))
        n2 = int(ceil(norm(v2)/0.2))
        write_trans_file(v1,v2,n1,n2)

        print(hkl_intlist)
        self.calc_condition.generate_files(axis=axis,hkl=hkl_intlist,xhi=xhi)


        os.chdir(f"{basedir}")
        
    @staticmethod    
    def order_indice(test):
        test = np.array(test)
        test = np.sort(test)
        x, y, z = test
        if x!=y:
            test = np.array([z,y,x])
        else:
            pass
        return test
    
    @staticmethod
    def get_100_angle(gbname):
        gbname = gbname.split("_")
        gbname = list(map(float, gbname))
        tang = min([float(gbname[1])/float(gbname[0]),float(gbname[0])/float(gbname[1])])
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        return mis_angle
    
    @staticmethod
    def get_110_angle(gbname):
        gbname = gbname.split("_")
        gbname = list(map(float, gbname))
        gbname = GBFactory.order_indice(gbname)
        x, y, z = gbname
        tang = z/np.sqrt(x**2 + y**2)
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        mis_angle = abs(mis_angle)  
        return mis_angle
    
    @staticmethod
    def get_111_angle(gbname):
        misangle_list = []
        baselist = []
        for i in list(itertools.permutations([1,-1,0])):
            baselist.append(list(i))     
        gbname = gbname.split("_")
        gbname = list(map(float, gbname))
        gbname = [-gbname[0],gbname[1],gbname[2]]
        misorientation_list = []
        for base in baselist:
            cos = dot(gbname,base)/(norm(gbname)*norm(base))
            mis_orientation = 2*(np.arccos(cos)/np.pi*180)
            mis_orientation =  abs(mis_orientation)
            misorientation_list.append(mis_orientation)
        mis_angle = min(misorientation_list)
        return mis_angle
    
    def get_gbilst(self):
        hkl_list = list(self.gb_dict.keys())
        gblist = Grainboundary_list()
        for each_hkl in hkl_list:
            if self.axis==[1,0,0]:
                tilt_angle = GBFactory.get_100_angle(each_hkl)
            elif self.axis==[1,1,0]:
                tilt_angle = GBFactory.get_110_angle(each_hkl)
            else:
                tilt_angle = GBFactory.get_111_angle(each_hkl)
            
            gblist.resister(hkl=each_hkl,tilt_angle=tilt_angle)
        
        self.gblist = gblist

In [37]:
def get_100_angle(gbname):
    gbname = gbname.split("_")
    gbname = list(map(float, gbname))
    tang = min([float(gbname[1])/float(gbname[0]),float(gbname[0])/float(gbname[1])])
    mis_angle = 2*(np.arctan(tang)/np.pi*180)
    return mis_angle

In [38]:
def get_110_angle(gbname):
    gbname = gbname.split("_")
    gbname = list(map(float, gbname))
    gbname = GBfactory.order_indice(gbname)
    x, y, z = gbname
    tang = z/np.sqrt(x**2 + y**2)
    mis_angle = 2*(np.arctan(tang)/np.pi*180)
    mis_angle = abs(mis_angle)  
    return mis_angle

In [39]:
def get_111_angle(gbname):
    misangle_list = []
    baselist = []
    for i in list(itertools.permutations([1,-1,0])):
        baselist.append(list(i))     
    gbname = gbname.split("_")
    gbname = list(map(float, gbname))
    gbname = [-gbname[0],gbname[1],gbname[2]]
    misorientation_list = []
    for base in baselist:
        cos = dot(gbname,base)/(norm(gbname)*norm(base))
        mis_orientation = 2*(np.arccos(cos)/np.pi*180)
        mis_orientation =  abs(mis_orientation)
        misorientation_list.append(mis_orientation)
    mis_angle = min(misorientation_list)
    return mis_angle

In [40]:
class Bcc:
    @staticmethod
    def vanishing_rule(hkl):
        print(hkl)
        x, y, z = hkl
        if ((x + y + z)%2==1):
            one_two = 2
        else:
            one_two = 1
        return one_two

In [41]:
class Crystal_structure:
    def __init__(self,vanishing_rule):
        self.vanishing_rule = vanishing_rule

In [42]:
bcc = Crystal_structure(Bcc.vanishing_rule)

In [43]:
class Fcc:
    @staticmethod
    def vanishing_rule(hkl):
        x, y, z = hkl
        if ((x%2==1) & (y%2==1) & (z%2==1)):
            one_two = 1    
        else:
            one_two = 2
        return one_two

In [44]:
fcc = Crystal_structure(Fcc.vanishing_rule)

In [45]:
def read_atominfile(filename):
    #read atom positions from POSCAR
    with open (f"{filename}",'r') as f:
        lines = f.readlines()
    atoms = np.array([0,0,0,0,0])
    for i in range(13,len(lines)):
        atoms = np.vstack((atoms,np.array(lines[i].split(),dtype = float)))
    atoms = np.delete(atoms,0,axis= 0)
    a = float(lines[5].split()[1])
    b = float(lines[6].split()[1])
    c = float(lines[7].split()[1])
    return a, b, c, atoms

In [46]:
# 電荷とatom typeはMgO向けに+2.0, -2.0にしている
def add_charge(atoms,a,b,c,type_list,charge_list):
    atoms_id = atoms[:,0]
    atoms_id = np.array(atoms_id,dtype=int)
    atoms_id_str = np.array(atoms_id,dtype=str)

    atoms_type = atoms[:,1]
    atoms_type = np.array(atoms_type,dtype=int)
    atoms_type_str = np.array(atoms_type,dtype=str)

    atoms_charge = np.zeros((len(atoms_type)))
    atoms_charge_str = np.array(atoms_charge,dtype=str)
    for type_num, atoms_charge in zip(atoms_type_list,atoms_charge_list):
#         print(atoms_charge_str[atoms_type_str==type_num])
        atoms_charge_str[atoms_type_str==type_num]=atoms_charge
#     atoms_charge_str[atoms_type_str=="1"] = "+2.0"
#     atoms_charge_str[atoms_type_str=="2"] = "-2.0"

    atoms_coor = atoms[:,2:]
    atoms_coor_str = np.array(atoms_coor,dtype=str)

    final_mat = np.hstack([atoms_id_str.reshape(-1,1),atoms_type_str.reshape(-1,1),atoms_charge_str.reshape(-1,1),atoms_coor_str])

    final_text = "\n".join([" ".join(i) for i in final_mat.tolist()])
    return final_text

In [47]:
def add_charge_to_atominfile(type_list, charge_list,filename="atominfile"):
    a, b, c, atoms = read_atominfile(f"{filename}")
    final_text = add_charge(atoms,a,b,c,type_list,charge_list)
    dim = np.array([1,1,1])
    X = atoms.copy()

    NumberAt = len(X) 

    dimx, dimy, dimz = dim

    xlo = 0.00000000
    xhi = a + 20
    ylo = 0.00000000
    yhi = b
    zlo = 0.00000000
    zhi = c

    yz = 0.0


    with open(filename, 'w') as f:
        f.write('#Header \n \n')
        f.write('{} atoms \n \n'.format(NumberAt))
        f.write('2 atom types \n \n')
        f.write('{0:.8f} {1:.8f} xlo xhi \n'.format(xlo, xhi))
        f.write('{0:.8f} {1:.8f} ylo yhi \n'.format(ylo, yhi))
        f.write('{0:.8f} {1:.8f} zlo zhi \n\n'.format(zlo, zhi))
        f.write('{0:.8f} {1:.8f} {2:.8f} xy xz yz \n\n'.format(0, 0, yz))            
        f.write('Atoms \n \n')
        f.write(f"{final_text}")
    f.close()

In [48]:
class SamplingMethodCreator_cnid_MEAM_energytot:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl
        
    def calc_condition(self,element):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        # 緩和条件
        relaxation = f"""
# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1 zero linear
fix fixbulk1 fixbulk1 setforce 0.0 0.0 0.0

velocity fixbulk2 zero linear
fix fixbulk2 fixbulk2 setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000
        """
        
        
        calc_condition = f"""
# define looping parameter------------------ 
include ./paras 

label loopa # a label where loop a starts
variable a loop ${{na}} # define a 'loop' variable
label loopb # a label where loop b starts
variable b loop ${{nb}} # define b 'loop' variable



variable ty equal "((v_a-1) * v_cnidv1y + (v_b-1) * v_cnidv2y)" # displacement in y direction
variable tz equal "((v_a-1) * v_cnidv1z + (v_b-1) * v_cnidv2z)" # displacement in z direction
variable tx equal 0 
# end end of defining looping parameter------------------



clear

#------------------Initialize Simulation ---------------------
kim init Sim_LAMMPS_Buckingham_SunStirnerHagston_2006_MgO__SM_152356670345_000 metal 
dimension 3 
boundary p p p
atom_style charge
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
kim interactions Mg O
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass_one}} # {element}
mass 2 ${{element_mass_two}} # {element}

#----------- grouping atoms into each element group --------------
group Mg type 1
group O type 2
group middleMg subtract middle Mg
group middleO subtract middle O


# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle reduce sum c_eng
compute emiddleMg middleMg reduce sum c_eng
compute emiddleO middleO reduce sum c_eng

compute bulk_dis bulk displace/atom 
compute hulk_dis_ave_x bulk reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in {element} crystal 
variable minimumenergy_one equal ${{minimumenergy_one}}
variable minimumenergy_two equal ${{minimumenergy_two}}
variable gbarea equal "ly * lz" 

# ---------- Run Minimization ---------------------
reset_timestep 0

displace_atoms right move 0 ${{ty}} ${{tz}} units box

{relaxation} 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle c_emiddleMg c_emiddleO temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
#variable esum equal "v_minimumenergy * count(middle)" 
variable esumMg equal "v_minimumenergy_one * count(middleMg)" 
variable esumO equal "v_minimumenergy_two * count(middleO)" 
variable esumMgall equal "v_minimumenergy_one * count(Mg)"
variable esumOall equal "v_minimumenergy_two * count(O)"
variable xsengMg equal "c_emiddleMg - (v_minimumenergy_one * count(middleMg))"
variable xsengO equal "c_emiddleO - (v_minimumenergy_two * count(middleO))"
variable gbe equal "(v_xsengMg + v_xsengO)/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable eng_tot equal "(c_eatoms - (v_esumMgall + v_esumOall))/v_gbarea"
variable e_tot equal ${{eng_tot}}
variable new_gbe equal ${{e_tot}}*16021.7733
variable num_middle equal "count(middle)"
variable num_middleMg equal "count(middleMg)"
variable num_middleO equal "count(middleO)"

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{tx}} ${{ty}} ${{tz}} ${{gbemJm2}} ${{new_gbe}}" append results


# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition

    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [49]:
nacl = Crystal_structure(Fcc.vanishing_rule)

In [50]:
atoms_type_list = ["1","2"]
atoms_charge_list = ["+2.0","-2.0"]

In [51]:
element = "MgO"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/kim/{element}"
MgO_kim = Element3(element,initializing_dir,nacl,atoms_type_list,atoms_charge_list)

In [52]:
fix_bulk_region = 50
middle_region = 50
start_bulk = 160
end_bulk = 165
vaccum = 20
cell_size = [100,20,20]

In [53]:
element_meam_list = [MgO_kim]

In [54]:
sampling_method = SamplingMethodCreator_cnid_MEAM_energytot
axis = [1,0,0]
max_sigma = 10

In [ ]:
for element_meam in element_meam_list:
    calc_condition = Calc_Condition(element=element_meam,SamplingMethod=sampling_method,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk,vaccum=vaccum)
    gbfactory = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition)
    for hkl in list(gbfactory.gb_dict.keys()):
        gbfactory.create(hkl,delivery_point=f"/homenfs1/yhata/calc/220728/{element_meam.name}")

In [2]:
class Grainboundary_list(object):
    def __init__(self):
        self._grainboundarylist = []
    
    # これがセッター
    def resister(self,hkl,tilt_angle):
        self._grainboundarylist.append(Grainboundary(hkl,tilt_angle))
        
    @property
    def grainboundarylist(self):
        return self._grainboundarylist
        
    @property
    def hkl(self):
        return np.array([i.hkl for i in self._grainboundarylist])
    
    @property
    def tilt_angle(self):
        return np.array([i.tilt_angle for i in self._grainboundarylist])

In [ ]:
Grainboundary = collections.namedtuple("Grainboundary",("hkl","tilt_angle"))

In [43]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self,element):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        # 緩和条件
        relaxation = f"""
# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000
        """
        

        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

# end of defining looping parameter------------------
# ------------- define parameter for each loop -------------

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"


clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam {element} {element}.meam {element}
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # {element}

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in {element} crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 

{relaxation}

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 


#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [59]:
class SamplingMethodCreator_cnid_MEAM_energytot:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl
        
    def calc_condition(self,element):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        # 緩和条件
        relaxation = f"""
# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1 zero linear
fix fixbulk1 fixbulk1 setforce 0.0 0.0 0.0

velocity fixbulk2 zero linear
fix fixbulk2 fixbulk2 setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000
        """
        
        
        calc_condition = f"""
# define looping parameter------------------ 
include ./paras 

label loopa # a label where loop a starts
variable a loop ${{na}} # define a 'loop' variable
label loopb # a label where loop b starts
variable b loop ${{nb}} # define b 'loop' variable



variable ty equal "((v_a-1) * v_cnidv1y + (v_b-1) * v_cnidv2y)" # displacement in y direction
variable tz equal "((v_a-1) * v_cnidv1z + (v_b-1) * v_cnidv2z)" # displacement in z direction
variable tx equal 0 
# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam {element} {element}.meam {element}
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # {element} 

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle reduce sum c_eng
compute bulk_dis bulk displace/atom 
compute hulk_dis_ave_x bulk reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in {element} crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 

# ---------- Run Minimization ---------------------
reset_timestep 0

displace_atoms right move 0 ${{ty}} ${{tz}} units box

{relaxation} 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{tx}} ${{ty}} ${{tz}} ${{gbemJm2}} ${{new_gbe}}" append results


# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition

    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [62]:
class SamplingMethodCreator_deletelayer_twist_energytot:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self,element):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        relaxation = f"""
# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000
        """
        
        
        sampling_points = int(self.squared_axis * self.one_two_axis)
        sampling_points = np.arange(sampling_points)
        sampling_points = itertools.combinations_with_replacement(sampling_points,2)
        sampling_points = np.array(list(map(list, list(sampling_points)))).T

        commands = np.array([["variable", "nr", "index"],
                            ["variable","nl","index"]])
        final_mat = np.hstack((commands,sampling_points))
        final_mat = "\n".join([" ".join(i) for i in final_mat.tolist()])
        n_layer = int(len(sampling_points.T))
        
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_axis equal sqrt(v_squared_axis)

variable n_layer equal {n_layer}

{final_mat}

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable

variable dx equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) * v_lattice_param"

variable tx_r equal "-(v_dx*(v_nr))" # displacement in x direction

variable tx_l equal "(v_dx*(v_nl))" # displacement in x direction

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam {element} {element}.meam {element}
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # {element}

displace_atoms right move ${{tx_r}} 0 0 units box
group left_dash region left 
group deleted_atoms_right subtract left_dash left
displace_atoms deleted_atoms_right move ${{startright}} 0 0 units box

displace_atoms left move ${{tx_l}} 0 0 units box
group right_dash region right
group deleted_atoms_left subtract right_dash right
displace_atoms deleted_atoms_left move -${{startright}} 0 0 units box

group fixbulk1_dash region fixbulk1 
group fixbulk2_dash region fixbulk2 
group middle_dash region middle 
group bulk_dash region bulk


# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in {element} crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


{relaxation}

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final_${{a}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{nl}} ${{nr}} ${{gbemJm2}} ${{new_gbe}}" append results

next nl
next nr
#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [ ]:

for element_meam in element_meam_list:
    calc_condition = Calc_Condition(element=element_meam,SamplingMethod=sampling_method,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)
    gbfactory = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition)
    for hkl in list(gbfactory.gb_dict.keys()):
        gbfactory.create(hkl,delivery_point=f"/homenfs1/yhata/calc/220712/{element_meam.name}")    